In [33]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization

hello there


In [34]:
def transform_resnet_block(nrOfFilters, inputLayer):
    init = keras.initializers.RandomNormal(stddev=0.02)
    resnet = keras.layers.Conv2D(nrOfFilters, (3,3), padding='same', kernel_initializer=init)(inputLayer)
    # instead of Batchnormalization
    resnet = InstanceNormalization(axis=-1)(resnet)
    resnet = keras.layers.Activation('relu')(resnet)
    # TODO reflection padding
    resnet = keras.layers.Conv2D(nrOfFilters, (3,3), padding='same', kernel_initializer=init)(resnet)
    resnet = InstanceNormalization(axis=-1)(resnet)
    resnet = keras.layers.Concatenate()([resnet, inputLayer])
    return resnet



In [35]:
# TODO generator model to use 6 ResNet layers as is used in the cycleGAN paper
def general_generator(imageShape=(256,256,3), nrOfResnet=9):
    init = keras.initializers.RandomNormal(stddev=0.02)
    inputImage = keras.Input(shape=imageShape)
    # ----Encode ------
    generator = keras.layers.Conv2D(64, (7,7), padding='same', kernel_initializer=init)(inputImage)
    generator = InstanceNormalization(axis=-1)(generator)
    generator = keras.layers.Activation('relu')(generator)

    generator = keras.layers.Conv2D(128, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(generator)
    generator = InstanceNormalization(axis=-1)(generator)
    generator = keras.layers.Activation('relu')(generator)

    generator = keras.layers.Conv2D(256, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(generator)
    generator = InstanceNormalization(axis=-1)(generator)
    generator = keras.layers.Activation('relu')(generator)

    # ----Transform -----
    for _ in range(nrOfResnet):
        generator = transform_resnet_block(256, generator)
    # ----Decode ------
    generator = keras.layers.Conv2DTranspose(128, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(generator)
    generator = InstanceNormalization(axis=-1)(generator)
    generator = keras.layers.Activation('relu')(generator)

    generator = keras.layers.Conv2DTranspose(64, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(generator)
    generator = InstanceNormalization(axis=-1)(generator)
    generator = keras.layers.Activation('relu')(generator)

    generator = keras.layers.Conv2D(3, (7,7), padding='same', kernel_initializer=init)(generator)
    generator = InstanceNormalization(axis=-1)(generator)
    outputImage = keras.layers.Activation('tanh')(generator)

    model = keras.Model(inputImage, outputImage)
    return model



In [36]:
model = general_generator()
model.summary()

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d_105 (Conv2D)             (None, 256, 256, 64) 9472        input_8[0][0]                    
__________________________________________________________________________________________________
instance_normalization_107 (Ins (None, 256, 256, 64) 128         conv2d_105[0][0]                 
__________________________________________________________________________________________________
activation_59 (Activation)      (None, 256, 256, 64) 0           instance_normalization_107[0][0] 
____________________________________________________________________________________________

In [37]:
from keras.optimizers import Adam
from keras.initializers import RandomNormal
from keras.models import Model
from keras.models import Input
from keras.layers import Conv2D
from keras.layers import LeakyReLU
from keras.layers import Activation
from keras.layers import Concatenate
from keras.layers import BatchNormalization
from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization

def general_discriminator(imageShape):
    init = RandomNormal(stddev=0.02)
    inputImage = Input(shape=imageShape)

    discriminator = Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(inputImage)
    discriminator = LeakyReLU(alpha=0.2)(discriminator)

    discriminator = Conv2D(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(discriminator)
    discriminator = InstanceNormalization(axis=-1)(discriminator)
    discriminator = LeakyReLU(alpha=0.2)(discriminator)

    discriminator = Conv2D(256, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(discriminator)
    discriminator = InstanceNormalization(axis=-1)(discriminator)
    discriminator = LeakyReLU(alpha=0.2)(discriminator)

    discriminator = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(discriminator)
    discriminator = InstanceNormalization(axis=-1)(discriminator)
    discriminator = LeakyReLU(alpha=0.2)(discriminator)

    discriminator = Conv2D(512, (4,4), padding='same', kernel_initializer=init)(discriminator)
    discriminator = InstanceNormalization(axis=-1)(discriminator)
    discriminator = LeakyReLU(alpha=0.2)(discriminator)

    output = Conv2D(1, (4,4), padding='same', kernel_initializer=init)(discriminator)
    model = Model(inputImage, output)
    # TODO negative log likelihood (e.g. binary cross entropy).
    model.compile(loss='mse', optimizer=Adam(learning_rate=0.0002, beta_1=0.5), loss_weights=[0.5])
    return model


In [38]:
imageShape = (256,256, 3)
model = general_discriminator(imageShape)
model.summary()

Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
conv2d_127 (Conv2D)          (None, 128, 128, 64)      3136      
_________________________________________________________________
leaky_re_lu_15 (LeakyReLU)   (None, 128, 128, 64)      0         
_________________________________________________________________
conv2d_128 (Conv2D)          (None, 64, 64, 128)       131200    
_________________________________________________________________
instance_normalization_131 ( (None, 64, 64, 128)       256       
_________________________________________________________________
leaky_re_lu_16 (LeakyReLU)   (None, 64, 64, 128)       0         
_________________________________________________________________
conv2d_129 (Conv2D)          (None, 32, 32, 256)       5245

In [41]:
# Mean squared error, lambda 1
def adversarial_loss_model(generator, discriminator, imageShape):
    inputGenerator = Input(shape=imageShape)
    generatorOutput = generator(inputGenerator)
    outputDiscriminator = discriminator(generatorOutput)
    model = Model(inputGenerator, outputDiscriminator)
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='mse', loss_weights=[1], optimizer=opt)
    return model

# If something already looks like from the target domain, you should not map it into a different image.
#Identity loss says that, if you fed image Y to generator G, it should yield the real image Y or something close to image Y.
# Mean absolute error, lambda 5
def identity_loss_model(generator, imageShape):
    inputIdentity = Input(shape=imageShape)
    outputIdentity = generator(inputIdentity)
    model = Model(inputIdentity, outputIdentity)
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='mae', loss_weights=[5], optimizer=opt)
    return model

# Mean absolute error, lambda 10
def forward_cycle_loss_model(generatorG, generatorF, imageShape):
    inputGenerator = Input(shape=imageShape)
    generatorG_output = generatorG(inputGenerator)
    generatorF_output = generatorF(generatorG_output)
    model = Model(inputGenerator, generatorF_output)
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='mae', loss_weights=[10], optimizer=opt)
    return model

# Mean absolute error, lambda 10
def backward_cycle_loss_model(generatorG, generatorF, imageShape):
    inputGenerator = Input(shape=imageShape)
    generatorF_output = generatorF(inputGenerator)
    generatorG_output = generatorG(generatorF_output)
    model = Model(inputGenerator, generatorG_output)
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='mae', loss_weights=[10], optimizer=opt)
    return model


In [42]:
# test code ---
imageShape = (256,256, 3)
# generator G(x) -> y'
Gx_2_y = general_generator(imageShape)
# generator F(y') -> x'
Fy_2_x = general_generator(imageShape)
# discriminator D(y') -> real/fake painting?
Dy = general_discriminator(imageShape)
# discriminator D(x') -> real/fake landscape?
Dx = general_discriminator(imageShape)

adversarial_loss_model_G = adversarial_loss_model(Gx_2_y, Dy, imageShape)
identity_loss_model_G = identity_loss_model(Gx_2_y, imageShape)
forward_cycle_loss_model_G = forward_cycle_loss_model(Gx_2_y, Fy_2_x, imageShape)
backward_cycle_loss_model_G = backward_cycle_loss_model(Gx_2_y, Fy_2_x, imageShape)

adversarial_loss_model_F = adversarial_loss_model(Fy_2_x, Dx, imageShape)
identity_loss_model_F = identity_loss_model(Fy_2_x, imageShape)
forward_cycle_loss_model_F = forward_cycle_loss_model(Fy_2_x, Gx_2_y, imageShape)
backward_cycle_loss_model_F = backward_cycle_loss_model(Fy_2_x, Gx_2_y, imageShape)